In [52]:
spark

org.apache.spark.sql.SparkSession@11592b68

In [53]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [54]:
val spark = SparkSession.builder.appName("Spark ML").getOrCreate()

spark = org.apache.spark.sql.SparkSession@11592b68


org.apache.spark.sql.SparkSession@11592b68

# Data preparation

In [55]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("lab05_train.csv")
    //.drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 0                     
 Unnamed: 0                  | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP          

trainDf = [_c0: int, Unnamed: 0: int ... 116 more fields]


[_c0: int, Unnamed: 0: int ... 116 more fields]

In [56]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [57]:
trainDf = trainDf.drop(categoriesCols:_*).na.fill(0)

trainDf = [_c0: int, Unnamed: 0: int ... 103 more fields]


[_c0: int, Unnamed: 0: int ... 103 more fields]

In [58]:
val Array(trainDfTrain, trainDfTest) = trainDf.randomSplit(Array(0.8, 0.2))

trainDfTrain = [_c0: int, Unnamed: 0: int ... 103 more fields]
trainDfTest = [_c0: int, Unnamed: 0: int ... 103 more fields]


[_c0: int, Unnamed: 0: int ... 103 more fields]

# Train model

In [59]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [60]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_68ecea3c3e80


vecAssembler_68ecea3c3e80

In [61]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [_c0: int, Unnamed: 0: int ... 104 more fields]


[_c0: int, Unnamed: 0: int ... 104 more fields]

In [62]:
trainDfCleaned.show(1, 0, true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _c0                        | 0                                                                                                                                                                                                                                                                                                                                                                                                            
 Unnamed: 0                 | 333149                                                                                                            

In [106]:
// Train RandomForest
val randomForestModel = new RandomForestClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setSeed(42)
    .setNumTrees(100)
    .fit(trainDfCleaned)

randomForestModel = RandomForestClassificationModel (uid=rfc_943c0da3dbfd) with 100 trees


RandomForestClassificationModel (uid=rfc_943c0da3dbfd) with 100 trees

In [107]:
val predictions = randomForestModel.transform(
    vectorAssembler.transform(trainDfTest)
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|   0.0|(104,[0,1,2,22,36...|[92.9594913482484...|[0.92959491348248...|       0.0|
|   0.0|(104,[0,1,2,12,14...|[92.4159994809014...|[0.92415999480901...|       0.0|
|   1.0|(104,[0,1,2,3,20,...|[85.3420165342106...|[0.85342016534210...|       0.0|
|   0.0|(104,[0,1,2,4,10,...|[94.0923955960658...|[0.94092395596065...|       0.0|
|   0.0|(104,[0,1,2,4,10,...|[93.5439769755567...|[0.93543976975556...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [_c0: int, Unnamed: 0: int ... 107 more fields]


[_c0: int, Unnamed: 0: int ... 107 more fields]

In [108]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0----------------------------------------------
 probability | [0.9295949134824841,0.0704050865175159] 
only showing top 1 row



In [109]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [110]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [111]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|    0.0704050865175159|
|   0.07584000519098565|
|   0.14657983465789423|
|    0.0590760440393422|
|   0.06456023024443311|
+----------------------+
only showing top 5 rows



In [112]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [113]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_ea65ab3fcf8c
accuracy = 0.7490116940453784


0.7490116940453784

# Predict

In [114]:
val testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("lab05_test.csv")
    .na.fill(0)

testDfData = [_c0: int, Unnamed: 0: int ... 115 more fields]


[_c0: int, Unnamed: 0: int ... 115 more fields]

In [115]:
testDfData.count()

44399

In [116]:
val realPredictions = randomForestModel.transform(vectorAssembler.transform(testDfData)).cache

realPredictions = [_c0: int, Unnamed: 0: int ... 119 more fields]


[_c0: int, Unnamed: 0: int ... 119 more fields]

In [117]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+-------------------+
|    ID|         prediction|
+------+-------------------+
|519130|0.11456214256126276|
|234045|0.06167705922671139|
|401256|0.06469617673196458|
|551070|0.11134230961901567|
|367285| 0.0688081208202768|
+------+-------------------+
only showing top 5 rows



In [118]:
realPredictions.coalesce(1)
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("prediction", "TARGET")
    .write
    .mode("overwrite")
    .format("csv")
    .option("header", "true")
    .option("delimiter","\t")
    .save("lab05.csv")

In [119]:
realPredictions.count()

44399

In [31]:
//spark.stop()

# Next steps

- add pipeline functionality `Pipeline().setStages(Array(..))`
- use categories (`StringIndexer`, `OneHotEncoder`)
- fill missing values with better data (not just zeros)
- play with hyper-parameters
- try another model (`LogisticRegression`, `GBTClassifier`, `MultilayerPerceptronClassifier`)
- use cross-validation (`TrainValidationSplit`)
